###Import Dependencies

In [ ]:
import json
import random
import numpy as np
from collections import Counter
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import scipy
from gensim.models.word2vec import Word2Vec

##Import Data

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training

Mounted at /content/drive
/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training


In [ ]:
#loading data
isolated_nodes = pd.read_csv("isolated_nodes.csv")
node_classification = pd.read_csv("node_classification.csv")      #there are 4 unique page types
training_graph = pd.read_csv("training_graph.csv")
with open('node_features_text.json') as f:
   node_features_text = json.load(f)

In [ ]:
#Changing integers to strings for Word2Vec input
listlist = []
for thing in node_features_text.values():
  listlist.append([str(i) for i in thing])

dict_values([[2835, 4518, 2198, 2014, 3818, 3045, 4570, 4585, 4596, 4599, 4609, 4617, 4627, 4641, 4643, 4651, 4665, 4676, 4682, 4694, 4696, 4708], [3293, 2534, 979, 2014, 2317], [4477, 2895, 4039, 3282, 765, 2669], [3133, 2052, 979, 3355, 107, 1878, 1105, 4571, 4582, 4593, 4598, 4609, 4619, 4629, 4636, 4642, 4652, 4662, 4675, 4683, 4689, 4704, 4713], [1151, 631, 2613, 2791, 648, 1684, 3818, 107, 1878, 1105, 4570, 4585, 4596, 4599, 4613, 4618, 4628, 4640, 4647, 4651, 4668, 4670, 4678, 4695, 4696, 4708], [3232, 4193, 296, 1337, 2832, 1437, 4572, 4587, 4590, 4605, 4613, 4615, 4632, 4634, 4643, 4659, 4660, 4673, 4686, 4687, 4700, 4713], [3383, 3832, 4328, 3939, 2332, 2566, 3244, 66], [2835, 2320, 3643, 2926, 3320, 1792, 3056, 4575, 4584, 4594, 4599, 4611, 4621, 4629, 4639, 4646, 4654, 4667, 4672, 4682, 4693, 4699, 4711], [1080, 1326, 2158, 2868, 4173, 2118, 4479, 448, 4575, 4583, 4592, 4597, 4612, 4622, 4630, 4637, 4650, 4658, 4662, 4676, 4684, 4689, 4698, 4708], [3251, 2620, 693, 216, 144

##Word2Vec Implementation

In [ ]:
word_2_vec_test = Word2Vec(listlist, min_count=1)
# print(word_2_vec_test)

Word2Vec(vocab=4714, size=100, alpha=0.025)


In [ ]:
wv = word_2_vec_test.wv

##Create np.array with averaged Word2Vec features for each node

In [ ]:
finlist = np.zeros((22470, 100))
for i in range(22470):
  n = len(node_features_text[str(i)])
  temp = np.zeros((n, 100))
  for j in range(n):
    temp[j,:] = wv[str(node_features_text[str(i)][j])]
  final = np.sum(temp, axis=0)/n
  finlist[i,:] = final

In [ ]:
df = pd.DataFrame(finlist)
#print(df.head)

##Import Node Classifications

In [ ]:
pages_df = pd.read_csv('node_classification.csv')
pages_df.drop(["id"], axis=1)
pages = np.array(pages_df)
pages = pages[:, 1]

##Take the element-wise squared differences of the features of every two connected nodes in the graph and put them in a data matrix

In [ ]:
edges = np.array(training_graph)
# print(len(edges))
subtracted_squared = np.zeros((len(edges), 101))
for i in range(len(edges)):
  temp = np.subtract(finlist[edges[i,0]], finlist[edges[i,1]])
  temp = np.square(temp)
  subtracted_squared[i,:100] = temp
  if pages[edges[i, 0]] == pages[edges[i,1]]:
    subtracted_squared[i, 100] = 0
  else:
    subtracted_squared[i, 100] = 1

##Create a list of unconnected nodes to add to the training data

In [ ]:
non_edges = np.zeros((len(edges),2), dtype=np.int64)
isolated = np.array(isolated_nodes)
i = 0
print(edges.shape)
while i < len(edges):
  temp = np.random.randint(0, 22470, size=(2,), dtype=np.int64)
  if temp[0] not in isolated:
    if temp[1] not in isolated:
      non_edges[i] = temp
      i += 1

##Take the element-wise squared differences of the features of every two nodes in the set we created of not connected nodes

In [ ]:
subtracted_squared_non = np.zeros((len(non_edges), 101))
for i in range(len(non_edges)):
  temp = np.subtract(finlist[non_edges[i,0]], finlist[non_edges[i,1]])
  temp = np.square(temp)
  subtracted_squared_non[i,:100] = temp
  if pages[non_edges[i, 0]] == pages[non_edges[i,1]]:
    subtracted_squared_non[i, 100] = 0
  else:
    subtracted_squared_non[i, 100] = 1

In [ ]:
non_edges_df = pd.DataFrame(non_edges)

##Create full data matrix with the squared differences pf connected and not connected node features

In [ ]:
X = np.zeros((len(edges)*2, 101))
X[:len(edges),:] = subtracted_squared
X[len(edges):,:] = subtracted_squared_non
y = np.zeros((len(edges)*2,), dtype=np.int8)
y[len(edges):] = 1

##Import dependencies for KNN model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

##Import test edges and labels

In [131]:
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Test
test_edges_df = pd.read_csv('test_edges.csv')
test_labels_df = pd.read_csv('test_labels.csv')
%cd /content/drive/Shareddrives/Datathon\ 2022\ -\ RichB\ Fan\ Club/Data/Training
test_edges = np.array(test_edges_df)

/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Test
/content/drive/Shareddrives/Datathon 2022 - RichB Fan Club/Data/Training


##Randomly split up data into test and training sets

In [138]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

##Test KNN model

In [ ]:
# create KNN classifier
knn = KNeighborsClassifier(n_neighbors=9, weights='distance')

# train the model using the training sets
knn.fit(X_train, y_train)

# predict the response for test dataset
y_pred = knn.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8618032414419873


##Perform element-wise subtraction and squaring on test edges

In [153]:
subtracted_squared_test = np.zeros((len(test_edges), 101))
for i in range(len(test_edges)):
  temp = np.subtract(finlist[test_edges[i,0]], finlist[test_edges[i,1]])
  temp = np.square(temp)
  subtracted_squared_test[i,:100] = temp
  if pages[non_edges[i, 0]] == pages[non_edges[i,1]]:
    subtracted_squared_test[i, 100] = 0
  else:
    subtracted_squared_test[i, 100] = 1

In [ ]:
knn_test = KNeighborsClassifier(n_neighbors=9, weights='distance')
knn_test.fit(X_scaled,y)
predictions = knn.predict(subtracted_squared_test)

In [137]:
test_labels = np.array(test_labels_df)
print(test_labels.flatten())
metrics.accuracy_score(test_labels, predictions)

[1 0 0 ... 0 0 0]


0.4195445258384412

##Create Neural Network Model

In [ ]:
from tensorflow.keras.models import Sequential
import tensorflow as tf

def trainModel(model,epochs,optimizer):
    batch_size=128
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics='accuracy')
    return model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size)
model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(512,activation='relu',input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
print(model.summary())
model_history=trainModel(model=model,epochs=20,optimizer='adam')
model.predict(X_test)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               52224     
                                                                 
 dropout_16 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 256)               131328    
                                                                 
 dropout_17 (Dropout)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               32896     
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_23 (Dense)            (None, 64)               

array([[0.1266061 ],
       [0.01656088],
       [0.21840918],
       ...,
       [0.9985467 ],
       [0.96935093],
       [0.93404377]], dtype=float32)

##Test NN Model

In [154]:
nn_predictions = model.predict(subtracted_squared_test)
nn_predictions = np.round_(nn_predictions.flatten())
nn_predictions = nn_predictions.astype(np.int8)
metrics.accuracy_score(test_labels, nn_predictions)